In [ ]:
from keras import backend as K
tf.keras.backend.clear_session()
from sklearn.preprocessing import LabelEncoder

epochs = 4   # number of all epochs in training
folds = 3 # number of folds

def get_new_labels(y):
    y_new = LabelEncoder().fit_transform([''.join(str(l)) for l in y])
    return y_new

y_new = get_new_labels(y)
# Assuming X and y are your data and labels
num_folds = 10  # choose the desired number of folds
kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=123)

print(get_new_labels(y))
fold_histories = []  # To store the history objects for each fold
best_model = None
best_val_loss = float('inf')

for train_index, valid_index in kf.split(X_train, y_train.argmax(1)):
    print("Train size:", len(train_index))
    print("Validation size:", len(valid_index))

    X_train_epoch, X_valid_epoch = X_train[train_index], X_train[valid_index]
    y_train_epoch, y_valid_epoch = X_train[train_index], X_train[valid_index]
    train_gen_epoch = train_generator.flow(X_train_epoch, y_train_epoch, shuffle= False, batch_size= batch_size)
    train_gen_epoch = test_generator.flow(X_valid_epoch, y_valid_epoch, shuffle= False, batch_size= batch_size)

    model = createModel()
    with tf.device('/device:GPU:0'):
        tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True)))
        history = model.fit(
                x=train_gen,
                epochs=epochs,
                verbose=1, 
                validation_data= valid_gen,
                validation_steps= None, 
                shuffle= False)
        
        val_loss = history.history['val_loss']
        index_loss = np.argmin(val_loss)
        val_lowest = val_loss[index_loss]

        # Evalutate the model on the validation set
        test = model.evaluate(valid_gen)
        print(test)
        # if test < best_val_loss:
        #         best_val_loss = test
        #         best_model = model

        tf.keras.backend.clear_session()

    fold_histories.append(history)


In [ ]:
preds = model.predict_generator(test_gen)
y_pred = np.argmax(preds, axis=1)

#images, labels = next(train_gen)  # get a batch size samples from the generator

# plt.figure(figsize=(20, 20))

# for i in range(12):
#     plt.subplot(4, 4, i + 1)
#     image = images[i]
#     plt.imshow(image)
#     index = np.argmax(labels[i])  # get image index
#     class_name = categories[index]   # get class of image
#     plt.title(class_name, color='blue', fontsize=12)
#     plt.axis('off')
label_to_class = {i: class_name for i, class_name in enumerate(categories)}

y_true = []

# Create generator

# Collect true labels during generator iteration
for i in range(len(X_test) // batch_size):
    _, labels = next(test_gen)
    y_true.extend(labels)

# Convert one-hot encoded labels to integers
y_true_indices = np.argmax(y_true, axis=1)

# Ensure the length of y_true_indices matches the length of y_pred_indices
y_pred_indices = np.argmax(y_pred, axis=1)[:len(y_true_indices)]

# Confusion matrix
cm = confusion_matrix(y_true_indices, y_pred_indices)

plt.figure(figsize= (10, 10))
plt.imshow(cm, interpolation= 'nearest', cmap= plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()

tick_marks = np.arange(len(categories))
plt.xticks(tick_marks, categories, rotation= 45)
plt.yticks(tick_marks, categories)


thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment= 'center', color= 'white' if cm[i, j] > thresh else 'black')

plt.tight_layout()
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

plt.show()